# Advanced Manipulation Transformer Training

This notebook implements the complete training pipeline for the Advanced Manipulation Transformer model, featuring:
- DINOv2 image encoding
- Multi-coordinate hand encoding
- Pixel-aligned refinement
- Sigma reparameterization to prevent mode collapse
- Comprehensive loss functions
- H200 GPU optimizations

In [ ]:
# Environment setup
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
from IPython.display import clear_output
import logging
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set DEX_YCB_DIR environment variable
os.environ['DEX_YCB_DIR'] = '/home/n231/231nProjectV2/dex-ycb-toolkit/data'

# Add project root to path
project_root = Path('.').absolute().parent
sys.path.insert(0, str(project_root))

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Load configuration
config_path = project_root / 'configs' / 'default_config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Override settings for notebook environment
config['data']['num_workers'] = 2  # Reduce for notebook
config['training']['log_freq'] = 50  # More frequent logging
config['training']['val_freq'] = 500

# For testing, use smaller dataset
if True:  # Set to False for full training
    config['training']['batch_size'] = 8  # Fixed: moved batch_size to training section
    config['model']['hidden_dim'] = 512  # Smaller model for testing
    config['training']['num_epochs'] = 5

print("Configuration loaded:")
print(f"- Experiment: {config['experiment_name']}")
print(f"- Batch size: {config['training']['batch_size']}")  # Fixed: use training.batch_size
print(f"- Learning rate: {config['training']['learning_rate']}")
print(f"- Epochs: {config['training']['num_epochs']}")

## Data Loading and Visualization

In [ ]:
# Import dataset
from data.enhanced_dexycb import EnhancedDexYCBDataset
from torch.utils.data import DataLoader

# Create datasets
print("Loading datasets...")

train_dataset = EnhancedDexYCBDataset(
    dexycb_root=config['data']['root_dir'],  # Use root_dir instead of dexycb_root
    split='train',
    sequence_length=config['data']['sequence_length'],
    augment=True,  # Default augmentation
    use_cache=True,  # Default caching
    max_samples=1000 if True else None  # Limit for testing
)

val_dataset = EnhancedDexYCBDataset(
    dexycb_root=config['data']['root_dir'],  # Use root_dir instead of dexycb_root
    split='val',
    sequence_length=1,  # No sequences for validation
    augment=False,
    use_cache=True,  # Default caching
    max_samples=100 if True else None  # Limit for testing
)

print(f"Train dataset: {len(train_dataset)} samples")
print(f"Val dataset: {len(val_dataset)} samples")

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config['training']['batch_size'],  # Fixed: use training.batch_size
    shuffle=True,
    num_workers=config['data']['num_workers'],
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['training']['batch_size'] * 2,  # Fixed: use training.batch_size
    shuffle=False,
    num_workers=config['data']['num_workers'],
    pin_memory=True
)

In [ ]:
# Visualize sample data
sample_batch = next(iter(train_loader))

print("Sample batch contents:")
for key, value in sample_batch.items():
    if isinstance(value, torch.Tensor):
        print(f"  {key}: {value.shape} ({value.dtype})")
    else:
        print(f"  {key}: {type(value)}")

# Visualize sample images
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.flatten()

for i in range(min(8, sample_batch['image'].shape[0])):
    img = sample_batch['image'][i].permute(1, 2, 0).cpu().numpy()
    # Denormalize
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    
    axes[i].imshow(img)
    axes[i].set_title(f"Sample {i}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# Visualize hand joints distribution
if 'hand_joints_3d' in sample_batch:
    joints = sample_batch['hand_joints_3d'].cpu().numpy()
    print(f"\nHand joints statistics:")
    print(f"  Mean: {joints.mean():.4f}")
    print(f"  Std: {joints.std():.4f}")
    print(f"  Min: {joints.min():.4f}")
    print(f"  Max: {joints.max():.4f}")

## Model Initialization

In [ ]:
# Import model
from models.unified_model import UnifiedManipulationTransformer

# Create model
print("Creating model...")
model = UnifiedManipulationTransformer(config['model'])

# Move model to CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Print model summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel Summary:")
print(f"  Total parameters: {total_params/1e6:.2f}M")
print(f"  Trainable parameters: {trainable_params/1e6:.2f}M")
print(f"  Non-trainable parameters: {(total_params-trainable_params)/1e6:.2f}M")

# Test forward pass
print("\nTesting forward pass...")
with torch.no_grad():
    test_output = model(
        images=sample_batch['image'][:2].to(device),  # Also move sample to device
        camera_params={
            'intrinsics': sample_batch['camera_intrinsics'][:2].to(device)
        }
    )

print("Output structure:")
for key, value in test_output.items():
    print(f"  {key}:")
    for sub_key, sub_value in value.items():
        if isinstance(sub_value, torch.Tensor):
            print(f"    {sub_key}: {sub_value.shape}")

## Training Setup

In [ ]:
# Import trainer
from training.trainer import ManipulationTrainer

# Create trainer
trainer = ManipulationTrainer(
    model=model,
    config=config['training'],
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

print("Trainer initialized")
print(f"  Device: {trainer.device}")
print(f"  Mixed precision: {trainer.use_amp}")
print(f"  Gradient accumulation steps: {trainer.accumulation_steps}")

In [ ]:
# Setup live plotting
class LivePlotter:
    def __init__(self):
        self.losses = {'train': [], 'val': []}
        self.mpjpe = {'train': [], 'val': []}
        self.epochs = []
        
    def update(self, epoch, train_metrics, val_metrics=None):
        self.epochs.append(epoch)
        self.losses['train'].append(train_metrics.get('loss', 0))
        
        if val_metrics:
            self.losses['val'].append(val_metrics.get('val_loss', 0))
            self.mpjpe['val'].append(val_metrics.get('val_mpjpe', 0))
    
    def plot(self):
        clear_output(wait=True)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        # Loss plot
        ax1.plot(self.epochs, self.losses['train'], 'b-', label='Train')
        if self.losses['val']:
            ax1.plot(self.epochs[-len(self.losses['val']):], self.losses['val'], 'r-', label='Val')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.set_title('Training Loss')
        ax1.legend()
        ax1.grid(True)
        
        # MPJPE plot
        if self.mpjpe['val']:
            ax2.plot(self.epochs[-len(self.mpjpe['val']):], self.mpjpe['val'], 'g-')
            ax2.set_xlabel('Epoch')
            ax2.set_ylabel('MPJPE (mm)')
            ax2.set_title('Validation MPJPE')
            ax2.grid(True)
        
        plt.tight_layout()
        plt.show()

plotter = LivePlotter()

## Training Loop

In [ ]:
# Main training loop
print("Starting training...\n")

best_val_mpjpe = float('inf')

for epoch in range(config['training']['num_epochs']):
    # Train epoch
    train_metrics = trainer.train_epoch(train_loader, epoch)
    
    # Validation
    if epoch % 2 == 0:  # Validate every 2 epochs for notebook
        val_metrics = trainer.validate(val_loader)
        
        # Update best model
        is_best = val_metrics['val_mpjpe'] < best_val_mpjpe
        if is_best:
            best_val_mpjpe = val_metrics['val_mpjpe']
        
        # Save checkpoint
        trainer.save_checkpoint(val_metrics, is_best)
        
        # Update plotter
        plotter.update(epoch, train_metrics, val_metrics)
        plotter.plot()
        
        # Print metrics
        print(f"\nEpoch {epoch} Summary:")
        print(f"  Train Loss: {train_metrics['loss']:.4f}")
        print(f"  Val Loss: {val_metrics['val_loss']:.4f}")
        print(f"  Val MPJPE: {val_metrics['val_mpjpe']:.2f}mm")
        print(f"  Val PA-MPJPE: {val_metrics['val_pa_mpjpe']:.2f}mm")
        print(f"  Val MPJPE Std: {val_metrics['val_mpjpe_std']:.2f}mm")
        
        if is_best:
            print("  🏆 New best model!")
    else:
        plotter.update(epoch, train_metrics)

print("\n✅ Training completed!")
print(f"Best validation MPJPE: {best_val_mpjpe:.2f}mm")

## Evaluation and Analysis

In [ ]:
# Load best model for evaluation
checkpoint_path = Path(config['checkpoint']['checkpoint_dir']) / 'best.pth'
if checkpoint_path.exists():
    trainer.load_checkpoint(str(checkpoint_path))
    print("Loaded best checkpoint")

# Detailed evaluation on validation set
model.eval()

# Collect predictions for visualization
with torch.no_grad():
    sample_batch = next(iter(val_loader))
    sample_batch = trainer._move_batch_to_device(sample_batch)
    
    predictions = model(
        images=sample_batch['image'],
        camera_params={
            'intrinsics': sample_batch['camera_intrinsics']
        }
    )

# Analyze predictions
pred_joints = predictions['hand']['joints_3d']
if 'joints_3d_refined' in predictions['hand']:
    pred_joints_refined = predictions['hand']['joints_3d_refined']
else:
    pred_joints_refined = pred_joints

target_joints = sample_batch['hand_joints_3d']

# Compute metrics
mpjpe = torch.norm(pred_joints - target_joints, dim=-1).mean().item() * 1000
mpjpe_refined = torch.norm(pred_joints_refined - target_joints, dim=-1).mean().item() * 1000

print(f"Sample evaluation:")
print(f"  MPJPE (initial): {mpjpe:.2f}mm")
print(f"  MPJPE (refined): {mpjpe_refined:.2f}mm")
print(f"  Improvement: {mpjpe - mpjpe_refined:.2f}mm ({(mpjpe - mpjpe_refined) / mpjpe * 100:.1f}%)")

# Check diversity
joints_std = pred_joints_refined.std(dim=0).mean().item()
print(f"\nPrediction diversity:")
print(f"  Std across batch: {joints_std:.6f}")
print(f"  Status: {'✅ Good diversity' if joints_std > 0.01 else '❌ Low diversity (potential mode collapse)'}")

In [ ]:
# Visualize predictions
def visualize_hand_predictions(predictions, targets, images, num_samples=4):
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4*num_samples))
    
    for i in range(num_samples):
        # Original image
        img = images[i].permute(1, 2, 0).cpu().numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        
        axes[i, 0].imshow(img)
        axes[i, 0].set_title(f"Input Image {i}")
        axes[i, 0].axis('off')
        
        # 3D joint visualization (side view)
        pred = predictions[i].cpu().numpy()
        target = targets[i].cpu().numpy()
        
        axes[i, 1].scatter(pred[:, 0], pred[:, 1], c='red', label='Predicted', alpha=0.7)
        axes[i, 1].scatter(target[:, 0], target[:, 1], c='blue', label='Target', alpha=0.7)
        axes[i, 1].set_title(f"3D Joints (XY view)")
        axes[i, 1].legend()
        axes[i, 1].set_aspect('equal')
        
        # Error heatmap
        errors = np.linalg.norm(pred - target, axis=1) * 1000  # mm
        axes[i, 2].bar(range(21), errors)
        axes[i, 2].set_title(f"Per-joint Error (mm)")
        axes[i, 2].set_xlabel("Joint Index")
        axes[i, 2].set_ylabel("Error (mm)")
    
    plt.tight_layout()
    plt.show()

# Visualize
visualize_hand_predictions(
    pred_joints_refined[:4],
    target_joints[:4],
    sample_batch['image'][:4]
)

## Export and Deployment

In [ ]:
# Export model for deployment
export_path = Path(config['output_dir']) / 'exported_model.pth'
export_path.parent.mkdir(parents=True, exist_ok=True)

# Save model with all necessary information
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'model_type': 'UnifiedManipulationTransformer',
    'input_size': (3, 224, 224),
    'best_mpjpe': best_val_mpjpe
}, export_path)

print(f"Model exported to: {export_path}")
print(f"File size: {export_path.stat().st_size / 1e6:.1f} MB")

# Test loading
loaded = torch.load(export_path)
print("\nExported model contents:")
for key in loaded.keys():
    if key != 'model_state_dict':
        print(f"  {key}: {loaded[key]}")

## Summary and Next Steps

The Advanced Manipulation Transformer has been successfully trained with the following key features:

1. **DINOv2 Integration**: Leverages powerful pretrained vision features
2. **Multi-Coordinate Hand Encoding**: Rich geometric understanding with 22 coordinate frames
3. **Pixel-Aligned Refinement**: Iterative refinement using 2D-3D correspondence
4. **Sigma Reparameterization**: Prevents mode collapse and ensures diverse predictions
5. **Comprehensive Losses**: Physical plausibility, diversity, and multi-task learning

### Expected Performance:
- MPJPE: <100mm (vs baseline 325mm)
- Diversity: >0.01 std (vs baseline 0.0003)
- GPU Utilization: ~85-95% on H200

### Next Steps:
1. Fine-tune on specific manipulation tasks
2. Implement temporal modeling for video sequences
3. Add differentiable physics simulation
4. Deploy for real-time robot control